In [5]:
import torch
from birdseye.sensor import * 
from birdseye.actions import *
from birdseye.env import RFEnv
from birdseye.rl_common.util import scale_ob
from birdseye.rl_common.models import CNN, MLP
from birdseye.utils import build_plots

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
save_path = 'checkpoints'
checkpoint_num = 10000
model_state_dict, optim_state_dict = torch.load('{}/{}.checkpoint'.format(save_path, checkpoint_num))


sensor = Drone()
actions = SimpleActions()
env = RFEnv(sensor, actions)

dueling = True
atom_num = 1
ob_scale = 1
policy_dim = len(env.actions.action_space)
in_dim = (1, 100, 100)
network = CNN(in_dim, policy_dim, atom_num, dueling)

network.load_state_dict(model_state_dict)

Available Actions:
  ID, Values
   0   (-30, 1)
   1   (-30, 2)
   2   (0, 1)
   3   (0, 2)
   4   (30, 1)
   5   (30, 2)


<All keys matched successfully>

In [8]:
num_steps = 5000

o = env.reset()

for i in range(num_steps): 
    with torch.no_grad():
        ob = scale_ob(np.expand_dims(o, 0), device, ob_scale)
        q = network(ob)

        a = q.argmax(1).cpu().numpy()[0]
        print(a)
        o_, r, done, info = env.step(a)

        build_plots(env.true_state, env.pf.particles)
        print('reward = ',reward)

KeyboardInterrupt: 